# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 9:48AM,258436,19,ELI030123.1,Eli Lilly and Company,Released
1,Mar 1 2023 9:45AM,258435,10,0086464699,ISDIN Corporation,Released
2,Mar 1 2023 9:45AM,258435,10,0086464700,ISDIN Corporation,Released
3,Mar 1 2023 9:45AM,258435,10,0086464701,ISDIN Corporation,Released
4,Mar 1 2023 9:45AM,258435,10,0086465007,ISDIN Corporation,Released
5,Mar 1 2023 9:45AM,258435,10,0086465215,ISDIN Corporation,Released
6,Mar 1 2023 9:45AM,258435,10,0086465405,ISDIN Corporation,Released
7,Mar 1 2023 9:45AM,258435,10,0086464683,ISDIN Corporation,Released
8,Mar 1 2023 9:45AM,258435,10,0086464686,ISDIN Corporation,Released
9,Mar 1 2023 9:45AM,258435,10,0086464690,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,258432,Released,2
25,258433,Released,3
26,258434,Released,34
27,258435,Released,65
28,258436,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258432,NaN,2.0
258433,NaN,3.0
258434,NaN,34.0
258435,NaN,65.0
258436,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258268,0.0,50.0
258354,0.0,49.0
258356,0.0,1.0
258357,0.0,29.0
258368,8.0,5.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258268,0,50
258354,0,49
258356,0,1
258357,0,29
258368,8,5


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258268,0,50
1,258354,0,49
2,258356,0,1
3,258357,0,29
4,258368,8,5


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258268,,50
1,258354,,49
2,258356,,1
3,258357,,29
4,258368,8,5


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 9:48AM,258436,19,Eli Lilly and Company
1,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation
66,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation
100,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc."
156,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc."
206,Mar 1 2023 9:32AM,258433,10,"Methapharm, Inc."
209,Mar 1 2023 9:25AM,258431,10,ISDIN Corporation
220,Mar 1 2023 9:25AM,258432,10,"CLINUVEL, Inc."
222,Mar 1 2023 9:19AM,258429,12,"SD Head USA, LLC"
223,Mar 1 2023 9:18AM,258426,19,"Emersa Waterbox, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 1 2023 9:48AM,258436,19,Eli Lilly and Company,,1
1,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,,65
2,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,,34
3,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",,56
4,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",,50
5,Mar 1 2023 9:32AM,258433,10,"Methapharm, Inc.",,3
6,Mar 1 2023 9:25AM,258431,10,ISDIN Corporation,,11
7,Mar 1 2023 9:25AM,258432,10,"CLINUVEL, Inc.",,2
8,Mar 1 2023 9:19AM,258429,12,"SD Head USA, LLC",1,
9,Mar 1 2023 9:18AM,258426,19,"Emersa Waterbox, LLC",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 9:48AM,258436,19,Eli Lilly and Company,1,
1,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,65,
2,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,34,
3,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",56,
4,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",50,
5,Mar 1 2023 9:32AM,258433,10,"Methapharm, Inc.",3,
6,Mar 1 2023 9:25AM,258431,10,ISDIN Corporation,11,
7,Mar 1 2023 9:25AM,258432,10,"CLINUVEL, Inc.",2,
8,Mar 1 2023 9:19AM,258429,12,"SD Head USA, LLC",,1
9,Mar 1 2023 9:18AM,258426,19,"Emersa Waterbox, LLC",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 9:48AM,258436,19,Eli Lilly and Company,1,
1,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,65,
2,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,34,
3,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",56,
4,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",50,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 9:48AM,258436,19,Eli Lilly and Company,1.0,NaN
1,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,65.0,NaN
2,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,34.0,NaN
3,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",56.0,NaN
4,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",50.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 9:48AM,258436,19,Eli Lilly and Company,1.0,0.0
1,Mar 1 2023 9:45AM,258435,10,ISDIN Corporation,65.0,0.0
2,Mar 1 2023 9:45AM,258434,10,ISDIN Corporation,34.0,0.0
3,Mar 1 2023 9:34AM,258375,15,"Alliance Pharma, Inc.",56.0,0.0
4,Mar 1 2023 9:33AM,258268,15,"Person & Covey, Inc.",50.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3875997,211.0,14.0
12,516850,1.0,1.0
15,516643,106.0,0.0
16,258425,1.0,0.0
19,1033696,8.0,21.0
22,258417,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3875997,211.0,14.0
1,12,516850,1.0,1.0
2,15,516643,106.0,0.0
3,16,258425,1.0,0.0
4,19,1033696,8.0,21.0
5,22,258417,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,211.0,14.0
1,12,1.0,1.0
2,15,106.0,0.0
3,16,1.0,0.0
4,19,8.0,21.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,211.0
1,12,Released,1.0
2,15,Released,106.0
3,16,Released,1.0
4,19,Released,8.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,22
Status,,,,,,
Executing,14.0,1.0,0.0,0.0,21.0,0.0
Released,211.0,1.0,106.0,1.0,8.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,22
0,Executing,14.0,1.0,0.0,0.0,21.0,0.0
1,Released,211.0,1.0,106.0,1.0,8.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,22
0,Executing,14.0,1.0,0.0,0.0,21.0,0.0
1,Released,211.0,1.0,106.0,1.0,8.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()